In [ ]:
import os
import sys

In [ ]:
IN_COLAB='google.colab' in sys.modules

In [ ]:
if IN_COLAB:
    from google.colab import drive

    WORKING_FOLDER="/content/drive/MyDrive/unicamp/ia368v_dd/trabalho_final_UNICAMP-IR/artigo/unicamp_IR"

    drive.mount('/content/drive', force_remount=True)

    os.chdir(WORKING_FOLDER)

    !pip install faiss-gpu -q
else:
    WORKING_FOLDER="/mnt/0060f889-4c27-409b-b0de-47f5427515e3/unicamp/ia368v_dd/unicamp_IR/"

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.8 MB/s eta 0:00:00


In [ ]:
import pickle
import pandas as pd
import numpy as np
import glob
import faiss
import time

In [11]:
DATASET_1M="../../clueweb22-pt_10M_sample_fixed/clueweb22-pt_cleaned_massiveweb_1M_sample.tsv"
QUERIES="tests/all_queries.tsv"

In [ ]:
with open("retrievals/adav2_1M_docs_000_999.pkl", "rb") as input_file:
    search_results = pickle.load(input_file)

In [ ]:
search_results[0].shape

(200, 1000)

In [ ]:
search_results[0][0]

array([0.87446404, 0.8707615 , 0.87009037, 0.8693617 , 0.8670238 ,
       0.86605275, 0.86397755, 0.86390376, 0.8636614 , 0.8635649 ,
       0.86348253, 0.8628488 , 0.8627183 , 0.8626984 , 0.86255586,
       0.8619136 , 0.86100805, 0.8608008 , 0.8605357 , 0.8600721 ,
       0.8597275 , 0.8597132 , 0.85934913, 0.8590995 , 0.8584981 ,
       0.85830516, 0.85830396, 0.85811114, 0.8580457 , 0.85795206,
       0.8579117 , 0.85779536, 0.8576883 , 0.85761094, 0.8574385 ,
       0.8572968 , 0.8572319 , 0.85717404, 0.856485  , 0.8563272 ,
       0.8559331 , 0.8555101 , 0.8553634 , 0.8552878 , 0.85526776,
       0.85519695, 0.8550583 , 0.8549913 , 0.8549326 , 0.85491806,
       0.85488   , 0.85482883, 0.8548054 , 0.85456824, 0.8545272 ,
       0.85451394, 0.8543373 , 0.8541226 , 0.8538931 , 0.8538497 ,
       0.85380566, 0.85369873, 0.85360104, 0.85356504, 0.8535327 ,
       0.85349685, 0.85348237, 0.85347277, 0.853422  , 0.8528943 ,
       0.8526833 , 0.8523139 , 0.85228443, 0.85227627, 0.85212

In [ ]:
search_results[1][0]

array([ 26808, 417552, 133606, 683678, 299347, 114590, 982277, 702506,
       845727, 361392, 299348, 561469, 674381, 377569, 909560,   4212,
       773349, 703732, 612756, 897021, 727063, 934207, 752166, 419996,
       331196, 647909, 397399, 123572, 884404, 712410, 645086, 151178,
       659738, 320265, 852106, 984567, 765470, 583383, 303216,  13319,
       534870, 935893, 392125, 383034, 702507, 273529, 478215, 857432,
       458239, 219234, 417928, 509867, 114592, 808386, 563695, 752405,
       114595, 341634,  26809,  98737, 718097, 468606, 290176, 961539,
        26810, 450209, 161032, 959636,  92689, 260842, 626837, 776380,
        98468,  52726, 330451, 498988,  92690, 439090, 913352, 683679,
       323073, 882217, 558009, 622137, 561298, 159420, 852902, 111617,
        51952, 383741,  89588, 946036, 330450, 303172, 767772, 767771,
       528814, 214825,  20452, 678567, 767764, 611855, 263189, 604841,
       204960, 807126, 161031, 909559, 920585, 619417, 237848, 928522,
      

In [12]:
dataset_df = pd.read_csv(DATASET_1M, sep='\t', header=None, names=['clueweb_id', 'passage', 'num_tokens', 'colbertx_id'])

In [13]:
queries_df = pd.read_csv(QUERIES, sep="\t")

In [14]:
dataset_df

,clueweb_id,passage,num_tokens,colbertx_id
0,clueweb22-pt0000-00-00003_1,"Se você precisar de ajuda, visite o website na...",361,5
1,clueweb22-pt0000-00-00003_2,Qual a parada final de Sintra para Cascais ôni...,382,6
2,clueweb22-pt0000-00-00003_3,"O Casino Estoril fica localizado no Estoril, a...",255,7
3,clueweb22-pt0000-00-00003_5,Sintra para Lisboa Sintra para Óbidos Leiria P...,283,8
4,clueweb22-pt0000-00-00008_0,Iniciar a digitalização Iniciar a digitalizaçã...,293,11
...,...,...,...,...
999995,clueweb22-pt0002-45-01752_2,"Pararam totalmente suas vidas, reagiram parand...",337,9999960
999996,clueweb22-pt0002-45-01757_0,Recicla Eletro - Coleta Gratuita de Lixo Eletr...,386,9999967
999997,clueweb22-pt0002-45-01760_4,É importante falar de liberdade ao falar de Ro...,261,9999981
999998,clueweb22-pt0002-45-01762_5,Os artigos 2º e artigos 84 a 86 da Lei nº 13.1...,375,9999989


In [15]:
queries_df

,query_id,query
0,0,Qual a maior característica da flora brasileira?
1,1,Qual a maior característica da fauna brasileira?
2,2,Por que os países Guiana e Suriname não são fi...
3,3,Por que a Bolívia tem duas capitais?
4,4,Qual o que significa decolonialidade?
...,...,...
195,195,O que pode deixar a internet lenta?
196,196,Como deixar a internet rápida?
197,197,Quais as qualidades das roupas peruanas?
198,198,"O que foi o movimento ""Impressionismo""?"


In [42]:
complete_search_results = []

for i in range(search_results[1].shape[0]):

    print("Handling results for query {} — {}".format(i, queries_df.iloc[i]['query']))

    for j in range(search_results[1].shape[1]):
        query_passage ={}
        query_passage['query_id'] = i
        query_passage['query'] = queries_df.iloc[i]['query']

        passage_index = search_results[1][i][j]

        query_passage['clueweb_id'] = dataset_df.iloc[passage_index]['clueweb_id']
        query_passage['colbertx_id'] = dataset_df.iloc[passage_index]['colbertx_id']
        query_passage['position'] = j + 1
        query_passage['score'] = search_results[0][i][j]
        query_passage['passage'] = dataset_df.iloc[passage_index]['passage']

        complete_search_results.append(query_passage)

Handling results for query 0 — Qual a maior característica da flora brasileira?
Handling results for query 1 — Qual a maior característica da fauna brasileira?
Handling results for query 2 — Por que os países Guiana e Suriname não são filiados a Conmebol?
Handling results for query 3 — Por que a Bolívia tem duas capitais?
Handling results for query 4 — Qual o que significa decolonialidade?
Handling results for query 5 — O que é sistema Swift? 
Handling results for query 6 — Por que a seleção uruguaia ostenta 4 estrelas em seu escudo?
Handling results for query 7 — Qual a importância da Semana de Arte Moderna de 1922?
Handling results for query 8 — Por que Johann Sebastian Bach é considerado um revolucionário na história da música?
Handling results for query 9 — quais países europeus seguem o regime monarquista?
Handling results for query 10 — como é definido o pib de um país?
Handling results for query 11 — quais os critérios de definição dos monumentos intitulados maravilhas do mundo 

In [43]:
complete_search_results_df = pd.DataFrame(complete_search_results)

In [44]:
complete_search_results_df.groupby('query_id').count()

,query,clueweb_id,colbertx_id,position,score,passage
query_id,,,,,,
0,1000,1000,1000,1000,1000,1000
1,1000,1000,1000,1000,1000,1000
2,1000,1000,1000,1000,1000,1000
3,1000,1000,1000,1000,1000,1000
4,1000,1000,1000,1000,1000,1000
...,...,...,...,...,...,...
195,1000,1000,1000,1000,1000,1000
196,1000,1000,1000,1000,1000,1000
197,1000,1000,1000,1000,1000,1000


In [45]:
complete_search_results_df.to_csv("retrievals/adav2_1M_docs_000_999.tsv", sep="\t", index=False)

### Check execution log

In [21]:
with open("tests/execution_with_reverse_rev_2.1_sampled_50_queries_gpt-4-1106-preview_20240117.pkl", "rb") as input_file:
    logs = pickle.load(input_file)

In [22]:
logs.keys()

dict_keys(['args', 'evaluation_cache', 'reverse_mapping'])

In [23]:
list(logs['evaluation_cache'].keys())[0]

'Onde está localizada a Praça XV de Novembro?_clueweb22-pt0000-44-08794_4'

In [54]:
len(logs['evaluation_cache'].keys())

3674

In [24]:
complete_search_results_df['query'].unique()

array(['Qual a maior característica da flora brasileira?',
       'Qual a maior característica da fauna brasileira?',
       'Por que os países Guiana e Suriname não são filiados a Conmebol?',
       'Por que a Bolívia tem duas capitais?',
       'Qual o que significa decolonialidade?', 'O que é sistema Swift? ',
       'Por que a seleção uruguaia ostenta 4 estrelas em seu escudo?',
       'Qual a importância da Semana de Arte Moderna de 1922?',
       'Por que Johann Sebastian Bach é considerado um revolucionário na história da música?',
       'quais países europeus seguem o regime monarquista?',
       'como é definido o pib de um país?',
       'quais os critérios de definição dos monumentos intitulados maravilhas do mundo moderno?',
       'por que paises escandinavos possuem bandeiras tão semelhantes?',
       'Como o Brasil reagiu a epidemia de AIDS no fim do século XX?',
       'Quais os riscos do uso de cigarros eletrônicos?',
       'Qual a maior torcida de futebol do Brasil?

### Filter the sampled 50 questions

In [25]:
reference_df = pd.read_csv("tests/50-dataset-doccano-queries-passagens-by-BM25-mT5_with_query_id.tsv", sep="\t")

In [28]:
sampled_queries = reference_df['query'].unique()

In [29]:
sampled_queries.shape

(50,)

In [46]:
sampled_queries_search_results_df = complete_search_results_df.merge(pd.DataFrame(sampled_queries, columns=['query']))

In [47]:
sampled_queries_search_results_df.iloc[:10]

,query_id,query,clueweb_id,colbertx_id,position,score,passage
0,1,Qual a maior característica da fauna brasileira?,clueweb22-pt0001-46-05303_17,7028401,1,0.877872,http://www.klimanaturali.org/2014/03/biomas-e-...
1,1,Qual a maior característica da fauna brasileira?,clueweb22-pt0000-17-01771_1,827050,2,0.875344,O Brasil é um país conhecido mundialmente por ...
2,1,Qual a maior característica da fauna brasileira?,clueweb22-pt0000-17-01771_19,827062,3,0.875237,A fauna brasileira está entre as mais diversas...
3,1,Qual a maior característica da fauna brasileira?,clueweb22-pt0000-96-07278_111,4600797,4,0.873625,"codornas (Nothura minor), seriemas (Cariama cr..."
4,1,Qual a maior característica da fauna brasileira?,clueweb22-pt0000-05-02747_0,264425,5,0.872606,BIOMAS E FITOFISIONOMIAS BRASILEIROS: CARACTER...
5,1,Qual a maior característica da fauna brasileira?,clueweb22-pt0002-07-02323_1,9847927,6,0.869637,Seu território plano agrega cerca de duzentos ...
6,1,Qual a maior característica da fauna brasileira?,clueweb22-pt0001-85-06153_3,8826893,7,0.868005,"Cerca de 650 espécies de aves povoam a região,..."
7,1,Qual a maior característica da fauna brasileira?,clueweb22-pt0000-64-06285_35,3048259,8,0.867367,O Brasil é um país de grande extensão territor...
8,1,Qual a maior característica da fauna brasileira?,clueweb22-pt0000-87-13049_0,4174990,9,0.867365,Ecossistemas brasileiros: resumo sobre as cara...
9,1,Qual a maior característica da fauna brasileira?,clueweb22-pt0001-80-10436_9,8592446,10,0.867054,"A fauna é igualmente rica, com mais de 500 esp..."


In [48]:
sampled_queries_search_results_df.to_csv("retrievals/adav2_1M_docs_000_999_sampled_50_queries.tsv", sep="\t", index=False)

In [49]:
sampled_queries_search_results_top_10_df = sampled_queries_search_results_df.groupby("query_id").head(10)

In [50]:
sampled_queries_search_results_top_10_df.iloc[:12]

,query_id,query,clueweb_id,colbertx_id,position,score,passage
0,1,Qual a maior característica da fauna brasileira?,clueweb22-pt0001-46-05303_17,7028401,1,0.877872,http://www.klimanaturali.org/2014/03/biomas-e-...
1,1,Qual a maior característica da fauna brasileira?,clueweb22-pt0000-17-01771_1,827050,2,0.875344,O Brasil é um país conhecido mundialmente por ...
2,1,Qual a maior característica da fauna brasileira?,clueweb22-pt0000-17-01771_19,827062,3,0.875237,A fauna brasileira está entre as mais diversas...
3,1,Qual a maior característica da fauna brasileira?,clueweb22-pt0000-96-07278_111,4600797,4,0.873625,"codornas (Nothura minor), seriemas (Cariama cr..."
4,1,Qual a maior característica da fauna brasileira?,clueweb22-pt0000-05-02747_0,264425,5,0.872606,BIOMAS E FITOFISIONOMIAS BRASILEIROS: CARACTER...
5,1,Qual a maior característica da fauna brasileira?,clueweb22-pt0002-07-02323_1,9847927,6,0.869637,Seu território plano agrega cerca de duzentos ...
6,1,Qual a maior característica da fauna brasileira?,clueweb22-pt0001-85-06153_3,8826893,7,0.868005,"Cerca de 650 espécies de aves povoam a região,..."
7,1,Qual a maior característica da fauna brasileira?,clueweb22-pt0000-64-06285_35,3048259,8,0.867367,O Brasil é um país de grande extensão territor...
8,1,Qual a maior característica da fauna brasileira?,clueweb22-pt0000-87-13049_0,4174990,9,0.867365,Ecossistemas brasileiros: resumo sobre as cara...
9,1,Qual a maior característica da fauna brasileira?,clueweb22-pt0001-80-10436_9,8592446,10,0.867054,"A fauna é igualmente rica, com mais de 500 esp..."


In [51]:
sampled_queries_search_results_top_10_df.to_csv("retrievals/adav2_1M_docs_000_009_sampled_50_queries.tsv", sep="\t", index=False)

In [52]:
already_analized = 0

for i, row in sampled_queries_search_results_top_10_df.iterrows():
    if (row['query'] + "_" + row['clueweb_id']) in logs['evaluation_cache']:
        print("Already analized...")

        already_analized += 1

Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...


In [53]:
already_analized

219